Metrics to extract

from raw.flagstat:
total reads
unmapped (total minus mapped)
singletons
improper mp (total minus properly paired minus unmapped minus singletons)

from raw.cleaned.dupmark.flagstat:
all dups

from PE.nodup.flagstat:
total PPE
total PPdups (properly paired from raw.flagstat, minus total from PE.nudup.flagstat)


(some data on laptop in /Users/jlchang/Documents/jean/Broad/BTL/ChIPseq/paired_dev/basicMetrics.xlsx 

In [1]:
cd /btl/analysis/ChIPseq/paired_dev/20171121_test

In [2]:
pwd

/btl/analysis/ChIPseq/paired_dev/20171121_test


In [3]:
source /broad/software/scripts/useuse

In [4]:
reuse -q .samtools-1.5

In [5]:
reuse -q .java-jdk-1.8.0_121-x86-64

create bam of just paired -f 2

In [6]:
samtools view -f 2 -bh /btl/analysis/ChIPseq/paired/SSFDEV-310_Kapa_Abconc_Miseq/K562-27ac-CST_S7_PE/K562-27ac-CST_S7.raw.cleaned.bam > K562-27ac-CST_S7.raw.PPE.bam

create bam using -F 1804 -f 2 - is there a difference?samtools view -f 2 -bh /btl/analysis/ChIPseq/paired/SSFDEV-310_Kapa_Abconc_Miseq/K562-27ac-CST_S7_PE/K562-27ac-CST_S7.raw.cleaned.bam > K562-27ac-CST_S7.raw.PPE.bam

In [7]:
samtools view -F 1804 -f 2 -bh /btl/analysis/ChIPseq/paired/SSFDEV-310_Kapa_Abconc_Miseq/K562-27ac-CST_S7_PE/K562-27ac-CST_S7.raw.cleaned.bam > K562-27ac-CST_S7.raw.1804PPE.bam

In [8]:
samtools flagstat K562-27ac-CST_S7.raw.PPE.bam > K562-27ac-CST_S7.raw.PPE.flagstat

In [9]:
samtools flagstat K562-27ac-CST_S7.raw.1804PPE.bam > K562-27ac-CST_S7.raw.1804PPE.flagstat

In [10]:
more K562-27ac-CST_S7.raw.PPE.flagstat

4160360 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
4160360 + 0 mapped (100.00% : N/A)
4160360 + 0 paired in sequencing
2080180 + 0 read1
2080180 + 0 read2
4160360 + 0 properly paired (100.00% : N/A)
4160360 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [11]:
more K562-27ac-CST_S7.raw.1804PPE.flagstat

4160360 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
4160360 + 0 mapped (100.00% : N/A)
4160360 + 0 paired in sequencing
2080180 + 0 read1
2080180 + 0 read2
4160360 + 0 properly paired (100.00% : N/A)
4160360 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


now markDuplicates

In [12]:
java -Xmx4G -jar /seq/software/picard-public/2.14.0/picard.jar MarkDuplicates INPUT=K562-27ac-CST_S7.raw.PPE.bam OUTPUT=K562-27ac-CST_S7.raw.PPE.dupmark.bam METRICS_FILE=K562-27ac-CST_S7.raw.PPE.markdupMetrics VALIDATION_STRINGENCY=LENIENT ASSUME_SORTED=true REMOVE_DUPLICATES=false 

13:12:33.846 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/seq/software/picard-public/2.14.0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Tue Nov 21 13:12:33 EST 2017] MarkDuplicates INPUT=[K562-27ac-CST_S7.raw.PPE.bam] OUTPUT=K562-27ac-CST_S7.raw.PPE.dupmark.bam METRICS_FILE=K562-27ac-CST_S7.raw.PPE.markdupMetrics REMOVE_DUPLICATES=false ASSUME_SORTED=true VALIDATION_STRINGENCY=LENIENT    MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP=50000 MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=8000 SORTING_COLLECTION_SIZE_RATIO=0.25 TAG_DUPLICATE_SET_MEMBERS=false REMOVE_SEQUENCING_DUPLICATES=false TAGGING_POLICY=DontTag CLEAR_DT=true DUPLICATE_SCORING_STRATEGY=SUM_OF_BASE_QUALITIES PROGRAM_RECORD_ID=MarkDuplicates PROGRAM_GROUP_NAME=MarkDuplicates READ_NAME_REGEX=<optimized capture of last three ':' separated fields as numeric values> OPTICAL_DUPLICATE_PIXEL_DISTANCE=100 MAX_OPTICAL_DUPLICATE_SET_SIZE=300000 ADD_PG_TAG_TO_READS=true VERBOSITY=INFO QUIET=false COMPRESSION_LEV

: 1

run above on lager instead (output indicates 
INFO	2017-11-21 13:14:37	MarkDuplicates	Marking 11276 records as duplicates.
INFO	2017-11-21 13:14:37	MarkDuplicates	Found 372 optical duplicate clusters.
INFO	2017-11-21 13:14:37	MarkDuplicates	Reads are assumed to be ordered by: coordinate)


Number of duplicates matches the number of reads removed in PE.nodup.bam after accounting for all "improperly paired" reads

In [13]:
samtools flagstat K562-27ac-CST_S7.raw.PPE.dupmark.bam > K562-27ac-CST_S7.raw.PPE.dupmark.flagstat

Experiment: should try markdup after sorting by name (documentation says coordinate-sorted won't mark both ends)

In [15]:
samtools view -f 2 -bh /btl/analysis/ChIPseq/paired/SSFDEV-310_Kapa_Abconc_Miseq/K562-27ac-CST_S7_PE/K562-27ac-CST_S7.raw.cleaned.bam | samtools sort -n - -T 'sorted_temp' -o K562-27ac-CST_S7.raw.PPE.nmsrt.bam

[bam_sort_core] merging from 2 files...


run below on lager

java -Xmx4G -jar /seq/software/picard-public/2.14.0/picard.jar MarkDuplicates INPUT=K562-27ac-CST_S7.raw.PPE.nmsrt.bam OUTPUT=K562-27ac-CST_S7.raw.PPE.nmsrt.dupmark.bam METRICS_FILE=K562-27ac-CST_S7.raw.PPE.markdupMetrics VALIDATION_STRINGENCY=LENIENT REMOVE_DUPLICATES=false 

Fail to test namesort effect (maybe need fixmate first?)
INFO	2017-11-21 13:27:19	MarkDuplicates	Reads are assumed to be ordered by: queryname
[Tue Nov 21 13:27:19 EST 2017] picard.sam.markduplicates.MarkDuplicates done. Elapsed time: 0.79 minutes.
Runtime.totalMemory()=3424124928
To get help, see http://broadinstitute.github.io/picard/index.html#GettingHelp
Exception in thread "main" java.lang.IllegalArgumentException: Alignments added out of order in SAMFileWriterImpl.addAlignment for file:///btl/analysis/ChIPseq/paired_dev/20171121_test/K562-27ac-CST_S7.raw.PPE.nmsrt.dupmark.bam. Sort order is queryname. Offending records are at [M03102:433:000000000-BC6GJ:1:1101:2856:9964] and [M03102:433:000000000-BC6GJ:1:1101:2856:10069]
	at htsjdk.samtools.SAMFileWriterImpl.assertPresorted(SAMFileWriterImpl.java:213)
	at htsjdk.samtools.SAMFileWriterImpl.addAlignment(SAMFileWriterImpl.java:200)
	at picard.sam.markduplicates.MarkDuplicates.doWork(MarkDuplicates.java:406)
	at picard.cmdline.CommandLineProgram.instanceMain(CommandLineProgram.java:268)
	at picard.cmdline.PicardCommandLine.instanceMain(PicardCommandLine.java:98)
	at picard.cmdline.PicardCommandLine.main(PicardCommandLine.java:108)

so few duplicates, probably not critical

In [16]:
more K562-27ac-CST_S7.raw.PPE.dupmark.flagstat

4160360 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
0 + 0 supplementary
11276 + 0 duplicates
4160360 + 0 mapped (100.00% : N/A)
4160360 + 0 paired in sequencing
2080180 + 0 read1
2080180 + 0 read2
4160360 + 0 properly paired (100.00% : N/A)
4160360 + 0 with itself and mate mapped
0 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


Pick high predicted duplication rate sample and check